In [2]:
from ChatPodcastGPT import *
import collections
import concurrent.futures
import os
import feedparser
import logging
import re
import tempfile
from PyPDF2 import PdfReader
from bs4 import BeautifulSoup
import requests
import retrying
import random
from IPython.display import Audio
import datetime

# MAX_TOKENS = 60_000 # GPT4-128k
MAX_TOKENS = 29_000
JOIN_NUM_DEFAULT = 300
# DEFAULT_TEXTGEN_MODEL = 'gpt-4-1106-preview'
DEFAULT_TEXTGEN_MODEL = 'GOOGLE/' + GoogleChat.MODELS['gemini-pro']
JINGLE_FILE_PATH = 'jazzstep.mp3'
with open(JINGLE_FILE_PATH, 'rb') as jingle_file:
    JINGLE_AUDIO = jingle_file.read()
JINGLE_AUDIO = JINGLE_AUDIO[:len(JINGLE_AUDIO) // 4]  # Shorten to just 4 sec

In [3]:
def clean_text(text):
    # Remove References Section
    text = re.sub(r'\bReferences?\b.*', '', text, flags=re.DOTALL|re.IGNORECASE)
    # Remove Footnotes Section
    text = re.sub(r'\bFootnotes?\b.*', '', text, flags=re.DOTALL|re.IGNORECASE)
    # Remove Figure/Table Insertions
    text = re.sub(r'- INSERT FIGURE \d AROUND HERE -', '', text)
    text = re.sub(r'- INSERT TABLE \d AROUND HERE -', '', text)
    # Remove extra whitespaces but keep line breaks
    text = re.sub(r'^\s+|\s+$', '', text, flags=re.MULTILINE)
    # Replace multiple spaces/newlines with a single space
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'\n+', '\n', text)
    return text

In [4]:
class PDFEpisode(Episode):
    PDFPart = collections.namedtuple('PDFPart', 'title text')

    def __init__(self, title, model=DEFAULT_TEXTGEN_MODEL, **kwargs):
        self.title = title
        self.model = model
        self.topic = kwargs.pop('topic', self.title) or self.title
        self._kwargs = kwargs
        self.join_num = JOIN_NUM_DEFAULT
        if 'podcast_args' in self._kwargs: self._kwargs.pop('podcast_args')
        super().__init__(topic=self.topic, **kwargs)

    def parse_pdf(self, file):
        """Parse a PDF and extract the text."""
        with open(file, "rb") as f:
            pdf = PdfReader(f)
            txt = clean_text('\n'.join(page.extract_text() for page in pdf.pages))
            if len(txt) < 1000: raise Exception("Text too small, cleaner must have messed up.")
            return txt

    def split_into_parts(self, text, max_tokens=MAX_TOKENS):
        """Split the text into parts based on titles and tokens."""
        lines = text.split("\n")
        parts = []
        current_part = []
        current_title = 'Paper'
        for line in lines:
            current_part.append(line)

            while Chat.num_tokens_from_text('\n'.join(current_part)) > max_tokens:
                part_text = '\n'.join(current_part)
                shortened_part, current_part = part_text[:max_tokens * 2], [part_text[max_tokens * 2:]]
                logger.info("PartAdd1")
                parts.append(self.PDFPart(current_title, shortened_part))

        if current_part:
            logger.info("PartAdd2")
            parts.append(self.PDFPart(current_title, "\n".join(current_part)))
        return parts

    def process_pdf(self, pdf_path):
        text = self.parse_pdf(pdf_path)
        parts = self.split_into_parts(text)
        return parts

    @retrying.retry(stop_max_attempt_number=3, wait_fixed=2000)
    def write_one_part(self, chat_msg, with_commercial=False):
        extra_system = """Explain the paper completely, in full verbose detail, as a single response.
Assume the listener doesn't know anything.
Follow this guide:

Introduction (500+ words)
Contextual background: Why this paper is significant in its field.
Key Concepts and Background: Explanation of the main scientific concepts or theories addressed in the paper.
(Optional) Breakdown of complex vocabulary used.

Core (2500+ words)
Detailed discussion of the research paper’s objectives.
Methodology and techniques used.
Key findings and results.

Implications and Applications (500+ words)
Analysis of the potential impact of these findings on the field.

Conclusion (500+ words)
Recap of the main points discussed in the episode.
Personal reflections on the paper and its broader relevance."""
        chat = PodcastChat(**{**self._kwargs, 'topic': self.title, 'extra_system': extra_system})
        msg, aud = chat.step(msg=chat_msg, model=self.model, ret_aud=True)
        chat._history.pop(2)
        chat._history[0]['content'] = chat._history[0]['content'][:len(chat._history[0]['content']) - len(extra_system)]
        com_msg, com_aud = chat.step(msg="Generate a funny, weird, and concise commercial for a company that now exists as a result of this paper.", model=self.model, ret_aud=True)
        msg = '\n'.join([msg, com_msg])
        try:
            aud = merge_mp3s([aud, JINGLE_AUDIO, com_aud])
        except Exception as e:
            logger.exception(e)
            raise
        return msg, aud

    def step(self):
        outline = self.data[0].text

        # Get parts
        with concurrent.futures.ThreadPoolExecutor(max_workers=16) as tpe:
            jobs = ([
                tpe.submit(self.write_one_part, f"""Explain the paper \"{self.title}\" completely, in full verbose detail.
Respond with the hosts names before each line like {self.chat._hosts[0]}: and {self.chat._hosts[1]}:
The text in the paper is:
{part.text}
""", with_commercial=True)
                for part in self.data
            ])
            job2idx = {j:i for i, j in enumerate(jobs)}
            self.sounds, self.summary_texts = [None] * len(jobs), [None] * len(jobs)
            for i, job in enumerate(concurrent.futures.as_completed(jobs)):
                logger.info(f"Part: {i} / {len(jobs)} = {100.0*i/len(jobs):,.5f}%")
                jobid = job2idx[job]
                text, aud = job.result()
                self.summary_texts[jobid] = text
                self.sounds[jobid] = aud

        return outline, '\n'.join(self.summary_texts)

In [5]:
class ArxivEpisode(PDFEpisode):
    ArxivPart = collections.namedtuple('ArxivPart', 'title text')

    def __init__(self, arxiv_id, id_is_url=False, title=None, model=DEFAULT_TEXTGEN_MODEL, **kwargs):
        self.arxiv_id = arxiv_id
        self.id_is_url = id_is_url
        self.title = title
        self.model = model
        self.data = self.process_pdf(self.arxiv_id)
        self.title = self.arxiv_title = self.get_title(self.arxiv_id)
        self._kwargs = kwargs
        super().__init__(title=self.arxiv_title, topic=self.arxiv_title, **kwargs)

    def split_into_parts(self, text, max_tokens=MAX_TOKENS):
        """Split the text into parts based on tokens."""
        lines = text.split("\n")
        parts = []
        current_part = [text]
        current_title = 'Paper'

        while Chat.num_tokens_from_text('\n'.join(current_part)) > max_tokens:
            part_text = '\n'.join(current_part)
            shortened_part, current_part = part_text[:max_tokens * 2], [part_text[max_tokens * 2:]]
            logger.info("PartAdd3")
            parts.append(self.ArxivPart(current_title, shortened_part))

        if current_part:
            logger.info(f"PartAdd4, {len(parts)}")
            parts.append(self.ArxivPart(current_title, "\n".join(current_part)))
        if len(parts) > 1:
            raise Exception("More than 1 part, giving up")
        return parts

    def process_pdf(self, arxiv_id):
        with tempfile.TemporaryDirectory() as tmpdir:
            file = os.path.join(tmpdir, "file.pdf")
            self.arxiv_download(arxiv_id, file)
            text = self.parse_pdf(file)
        parts = self.split_into_parts(text)
        return parts
    
    def arxiv_download(self, arxiv_id, out_file):
        if self.id_is_url:
            url = arxiv_id
        else:
            url = f"https://arxiv.org/pdf/{arxiv_id}.pdf"
        response = requests.get(url)
        with open(out_file, "wb") as f:
            f.write(response.content)

    def get_title(self, arxiv_id):
        if self.title is not None:
            return self.title
        url = f"https://arxiv.org/abs/{arxiv_id}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.find('h1', {'class': 'title mathjax'}).text.strip().split('\n')[-1].strip()
        return title

In [6]:
class CommercialGenerator:
    def get_random_company(self):
        chat = Chat("Return simple plaintext responses only.")
        with open("nouns.txt") as f:
            nouns = f.read().splitlines()
        random_noun = random.choice(nouns)
        return chat.message(f"Write just 1 funny, weird, creative made up company that doesn't exist involving {random_noun}.", temperature=1)

    def generate(self, company=None):
        if company is None:
            company = self.get_random_company()
        chat = PodcastChat(f"Very short commercial for {company}", host_voices=[OpenAITTS(OpenAITTS.MAN), OpenAITTS(OpenAITTS.WOMAN)])
        chat._history[-1] = {"role": "user", "content": f"Generate a very funny, weird, and short commercial for {company}, who is sponsoring the podcast."}
        return chat.step(model=DEFAULT_TEXTGEN_MODEL)

In [7]:
class ArxivRunner:
    def __init__(self, category, start=0, limit=5):
        self.category = category
        self.start = start
        self.limit = limit

    def get_top(self):
        """Retrieve top Arxiv entries based on category."""
        if self.category == 'psyarxiv': return self.get_top_psyarxiv()
        url = f'https://arxiv.org/list/{self.category}/recent'
        print(url)
        html = requests.get(url).content
        soup = BeautifulSoup(html, 'html.parser')
        articles = []
        
        for item in soup.find_all('dt'):
            title = item.find_next_sibling('dd').find('div', class_='list-title').text.replace('Title:', '').strip()
            identifier = item.find('span', class_='list-identifier').a.text
            pdf_link = 'https://arxiv.org' + item.find('span', class_='list-identifier').find('a', title='Download PDF')['href']
        
            articles.append({
                'title': title,
                'ID': identifier,
                'pdf': pdf_link
            })
        return [a["pdf"].split('/')[-1] for a in articles]

    def get_top_psyarxiv(self):
        url = 'https://share.osf.io/api/v3/index-card-search?cardSearchFilter%5BresourceType%5D=Preprint&cardSearchFilter%5Bpublisher%5D%5B%5D=https%3A%2F%2Fosf.io%2Fpreprints%2Fpsyarxiv&cardSearchFilter%5BaccessService%5D=https%3A%2F%2Fosf.io%2F&cardSearchText%5B*%2Ccreator.name%2CisContainedBy.creator.name%5D=&page%5Bcursor%5D=&page%5Bsize%5D=100&sort=-dateCreated'
        print(url)
        data = requests.get(url, headers={'Accept': 'application/vnd.api+json'}).json()
        data = [x for x in data['included'] if x["type"] == "index-card"]
        return [(f"{x['attributes']['resourceIdentifier'][0]}/download/", x['attributes']['resourceMetadata']['title'][0]['@value']) for x in data]

In [8]:
MODEL = DEFAULT_TEXTGEN_MODEL
HOST_VOICES = [OpenAITTS(OpenAITTS.MAN), OpenAITTS(OpenAITTS.WOMAN)]
# HOST_VOICES = [AWSPollyTTS(AWSPollyTTS.MAN), AWSPollyTTS(AWSPollyTTS.WOMAN)]
PODCAST_ARGS = ("ArxivPodcastGPT", "ArxivPodcastGPT.github.io", "podcasts/ComputerScience/Consolidated/podcast.xml")

def create_large_episode(arxiv_category, limit=5, add_commercials=False):
    """Create a podcast episode with Arxiv papers."""
    audios, texts = [JINGLE_AUDIO], []
    successes = 0
    
    for arxiv_id in ArxivRunner(arxiv_category, limit=limit).get_top():
        if successes >= limit:
            break

        arxiv_kwargs = {'id_is_url': False}
        if isinstance(arxiv_id, tuple):
            arxiv_id, arxiv_title = arxiv_id
            arxiv_kwargs['title'] = arxiv_title
            arxiv_kwargs['id_is_url'] = True
            
        logger.info(f"Trying arxiv ID {arxiv_id} in {arxiv_category} with {successes}/{limit}")
        try:
            arxiv_episode = ArxivEpisode(arxiv_id, model=MODEL, podcast_args=PODCAST_ARGS, host_voices=HOST_VOICES, **arxiv_kwargs)
            outline, txt = arxiv_episode.step()
            logger.info(f"Got outline: {outline[:500]}")
        except Exception as e:
            logger.exception(f"Error processing arxiv_id {arxiv_id}: {e}")
            continue

        audios.append(merge_mp3s(arxiv_episode.sounds))
        audios.append(JINGLE_AUDIO)
        arxiv_title = re.sub('[^0-9a-zA-Z]+', ' ', arxiv_episode.arxiv_title)
        texts.append(f'ChatGPT generated podcast using model={MODEL} for https://arxiv.org/abs/{arxiv_id} {arxiv_title}')
        successes += 1
        logger.info(texts[-1])

        if not add_commercials:
            continue
        try:
            commercial_text, commercial_sound = CommercialGenerator().generate()
            audios.append(commercial_sound)
            audios.append(JINGLE_AUDIO)
        except Exception as e:
            logger.error("Unable to generate commercial")
            logger.exception(e)
    
    return audios, texts

In [9]:
def get_title(texts):
    chat = Chat("Return just simple plaintext.")
    return chat.message(
        "Given the following papers, write a clickbait title that captures all of them. " + 
        ", ".join(txt.split(' Title ')[-1] for txt in texts)
    )

In [10]:
class AudioCompletedEpisode(Episode):
    def __init__(self, sounds, podcast_args):
        self.sounds = sounds
        self.pod = PodcastRSSFeed(*podcast_args)

In [11]:
arxiv_categories = ["AI", "CL", "CC", "CE", "CG", "GT", "CV", "CY", "CR", "DS", "DB", "DL", "DM", "DC", "ET", "FL", "GL", "GR", "AR", "HC", "IR", "IT", "LO", "LG", "MS", "MA", "MM", "NI", "NE", "NA", "OS", "OH", "PF", "PL", "RO", "SI", "SE", "SD", "SC", "SY"]

def run(arxiv_category, upload=True, limit=5):
    audios, texts = create_large_episode(arxiv_category, limit=limit)
    ep = AudioCompletedEpisode(audios, podcast_args=PODCAST_ARGS)
    if upload:
        ep.upload(f'{datetime.datetime.now():%Y-%m-%d} {arxiv_category}: {get_title(texts)}', '\n\n'.join(texts))
    return ep

In [13]:
# %%time
# ep = run("psyarxiv", upload=False, limit=1)
# IPython.display.Audio(merge_mp3s(ep.sounds))

https://share.osf.io/api/v3/index-card-search?cardSearchFilter%5BresourceType%5D=Preprint&cardSearchFilter%5Bpublisher%5D%5B%5D=https%3A%2F%2Fosf.io%2Fpreprints%2Fpsyarxiv&cardSearchFilter%5BaccessService%5D=https%3A%2F%2Fosf.io%2F&cardSearchText%5B*%2Ccreator.name%2CisContainedBy.creator.name%5D=&page%5Bcursor%5D=&page%5Bsize%5D=100&sort=-dateCreated
2023-12-14T16:55:03.581627Z [info     ] Trying arxiv ID https://osf.io/96wqp/download/ in psyarxiv with 0/1 [__main__] calling_func=create_large_episode thread_id=[Main]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in create_large_episode:23                                                                       │
│                                                                                                  │
│   20 │   │                                                                                       │
│   21 │   │   logger.info(f"Trying arxiv ID {arxiv_id} in {arxiv_category} with {successes}/{l    │
│   22 │   │   try:                                                                                │
│ ❱ 23 │   │   │   arxiv_episode = ArxivEpisode(arxiv_id, model=MODEL, podcast_args=PODCAST_ARG    │
│   24 │   │   │   outline, txt = arxiv_episode.step()                                             │
│   25 │   │   │   logger.info(f"Got outline: {outline[:500]}")                                    │
│   26 │   │   except Exception as e:                                                              │
│                                                                                                  │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │ add_commercials = False                                                                      │ │
│ │  arxiv_category = 'psyarxiv'                                                                 │ │
│ │        arxiv_id = 'https://osf.io/96wqp/download/'                                           │ │
│ │    arxiv_kwargs = {                                                                          │ │
│ │                   │   'id_is_url': True,                                                     │ │
│ │                   │   'title': 'Effector-dependent improvements in action prediction in      │ │
│ │                   left-handed individuals '+34                                               │ │
│ │                   }                                                                          │ │
│ │     arxiv_title = 'Effector-dependent improvements in action prediction in left-handed       │ │
│ │                   individuals '+34                                                           │ │
│ │          audios = [                                                                          │ │
│ │                   │                                                                          │ │
│ │                   b'\xff\xf3\x84\xc4\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0… │ │
│ │                   ]                                                                          │ │
│ │               e = PdfReadError('EOF marker not found')                                       │ │
│ │           limit = 1                                                                          │ │
│ │       successes = 0                                                                          │ │
│ │           texts = []                                                                         │ │
│ ╰──────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                  │
│ in __init__:9                                                                                    │
│                                                                                                  │
│    6 │   │   self.id_is_url = id_is_url                                                          │
│    7 │   │   self.title = title                                                                  │
│    8 │   │   self.model = model                                                                  │
│ ❱  9 │   │   self.data = self.process_pdf(self.arxiv_id)                                         │
│   10 │   │   self.title = self.arxiv_title = self.get_title(self.arxiv_id)                       │
│   11 │   │   self._kwargs = kwargs                         

2023-12-14T16:55:06.855467Z [error    ] Error processing arxiv_id https://osf.io/96wqp/download/: EOF marker not found [__main__] calling_func=error thread_id=[Main]

2023-12-14T16:55:07.241029Z [info     ] Trying arxiv ID https://osf.io/f9ukv/download/ in psyarxiv with 0/1 [__main__] calling_func=create_large_episode thread_id=[Main]
2023-12-14T16:55:13.079589Z [info     ] PartAdd4, 0                    [__main__] calling_func=split_into_parts thread_id=[Main]
https://raw.githubusercontent.com/JonathanGrant/jonathangrant.github.io/main/podcasts/podcast.xml
2023-12-14T16:55:13.288663Z [info     ] Currently at self.num_tokens_from_messages(self._history)=7804 tokens in conversation [__main__] calling_func=message thread_id=123145667891200 thread_name=ThreadPoolExecutor-0_0
2023-12-14T16:55:13.289955Z [info     ] requesting chatcompletion model='MISTRAL/mistral-medium'... [__main__] calling_func=_msg thread_id=123145667891200 thread_name=ThreadPoolExecutor-0_0
2023-12-14T16:55:57.913319Z

2023-12-14T16:56:06.464623Z [info     ] Currently at self.num_tokens_from_messages(self._history)=996 tokens in conversation [__main__] calling_func=message thread_id=123145667891200 thread_name=ThreadPoolExecutor-0_0
2023-12-14T16:56:06.465605Z [info     ] requesting chatcompletion model='MISTRAL/mistral-medium'... [__main__] calling_func=_msg thread_id=123145667891200 thread_name=ThreadPoolExecutor-0_0
2023-12-14T16:56:18.578815Z [info     ] received chatcompletion model='mistral-medium'... [__main__] calling_func=_msg thread_id=123145667891200 thread_name=ThreadPoolExecutor-0_0
2023-12-14T16:56:18.581224Z [info     ] requesting tts i=0 voice=<ChatPodcastGPT.AWSPollyTTS object at 0x1128c2b90> [__main__] calling_func=write_audio thread_id=123145684680704 thread_name=ThreadPoolExecutor-2_0
2023-12-14T16:56:20.401065Z [info     ] received tts i=0 voice=<ChatPodcastGPT.AWSPollyTTS object at 0x1128c2b90> [__main__] calling_func=write_audio thread_id=123145684680704 thread_name=ThreadPoolE

2023-12-14T16:56:24.842593Z [info     ] Part: 0 / 1 = 0.00000%         [__main__] calling_func=step thread_id=[Main]
2023-12-14T16:56:24.844818Z [info     ] Got outline: This work is shared under a CC BY-SA 4.0 license unless otherwise notedBlending simulation and abstraction for physical reasoning
Felix A. Sosa1,2∗Samuel J. Gershman1,2,3Tomer D. Ullman1,2
1Department of Psychology, Harvard University
2Center for Brains, Minds, and Machines, MIT
3Center for Brain Science, Harvard University
Abstract
How are people able to understand everyday physical events with such ease? One hypothesis
suggests people use an approximate probabilistic simulation of the world. A [__main__] calling_func=create_large_episode thread_id=[Main]
2023-12-14T16:56:27.744200Z [info     ] ChatGPT generated podcast using model=MISTRAL/mistral-medium for https://arxiv.org/abs/https://osf.io/f9ukv/download/ Blending simulation and abstraction for physical reasoning [__main__] calling_func=create_large_episode threa